
## Projeto | Construido uma rede com jogos da Steam 🎮️

    
---------------------

**Equipe**


 * Letícia Coêlho Barbosa       | [Github](https://github.com/leticiacb1)
 <br>

 * Lídia Alves Chagas Domingos  | [Github](https://github.com/LidiaDomingos)
 <br>

 * Lorran Caetano Lopes         | [Github](https://github.com/lorrancmlopes)
 <br>
 
 ---------------------

* **Dataset escolhido** : [Jogos da Steam](https://www.kaggle.com/datasets/fronkongames/steam-games-dataset)

### O DATASET

-------

O contexto do projeto engloba os jogos famosos para computadores disponíveis na Steam. Atualmente, a plataforma Steam possui aproximadamente 95 milhões de usuários no mundo todo e é sucesso entre os gamers, sendo a atual líder do mercado. 


#### Principais colunas
<br>

O dataset escolhido possui 39 colunas, nesse tópico citaremos a descrições de algumas colunas de maior interesse para analise.

* **name** : nome do jogo.
<br>

* **releaseDate** : data de lançamento.
<br>

* **price** : preço do jogo.
<br>

* **metacritic_score** : media das notas dadas por avaliações de profissionais.
<br>

* **user_score** : media das notas dadas pelos usuários do jogo.
<br>

* **score_rank** : o ranke do jogo baseado nas avaliações dos usuários.
<br>

* **developers** : desenvolvedores dos jogos.
<br>

* **categories** : categoria dos jogos.
<br>

* **genres** : genero do jogo.
<br>

* **tags** : Tags do jogo.
<br>

 
 ### NOSSA REDE

-------

* **Vértices** : Jogos da Steam.
<br>

* **Arestas**  : Existe uma aresta entre um jogo 1 e um jogo 2 se o número de tags em comum entre os jogos for superior ou igual a um **threadhold** (definido mais a frente)


 ### HIPOTESE

-------


`"Quanto mais central é a sua posição na rede, maior a média de avaliações dos críticos profissionais."`
<br>

* **Variável de controle**: Tamanho da empresa.
<br>

* **Variável dependente**:  medida indicativa de centralidade do nó na rede (coreness).
<br>

* **Variável independente**: dataset[Metacritic score]

### Analisando os dados 

-------

In [1]:
# ---------------------------------
# ------------ IMPORTS ------------
# ---------------------------------

import seaborn as sns
from netpixi.integration.gt import *
from regression.integration.gt import *
import netpixi
import pandas as pd

import numpy as np
import seaborn as sns
import regression as reg

from scipy import stats
from matplotlib import pyplot as plt

import cpnet
from graph_tool import spectral

<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


In [2]:
# Grafo não direcional | Informação obtida via documentação
g = Graph(directed=False) 

In [4]:
# ----  Lendo o arquivo ----

df_principal = pd.read_csv('games.csv')

columns = [col for col in df_principal.columns]

print(f"\n> Colunas : {columns}\n")

print(f"\n> Tamanho do dataset : {df_principal.shape}\n")

print(f"\n> Primeiras linhas dataset : \n")
df_principal.head(2)


> Colunas : ['AppID', 'Name', 'Release date', 'Estimated owners', 'Peak CCU', 'Required age', 'Price', 'DLC count', 'About the game', 'Supported languages', 'Full audio languages', 'Reviews', 'Header image', 'Website', 'Support url', 'Support email', 'Windows', 'Mac', 'Linux', 'Metacritic score', 'Metacritic url', 'User score', 'Positive', 'Negative', 'Score rank', 'Achievements', 'Recommendations', 'Notes', 'Average playtime forever', 'Average playtime two weeks', 'Median playtime forever', 'Median playtime two weeks', 'Developers', 'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies']


> Tamanho do dataset : (72934, 39)


> Primeiras linhas dataset : 



,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...




#### Descrevendo melhor a coluna Metacritic score

In [5]:
print(f"\n> Valores únicos presentes:\n\n {df_principal['Metacritic score'].unique()}\n")

print(f"\n> Distribuição dos valores :\n\n {df_principal['Metacritic score'].value_counts()}\n")

print(f"\n> Descrição:\n\n{df_principal['Metacritic score'].describe()}\n")


> Valores únicos presentes:

 [ 0 62 64 53 84 89 70 86 40 87 92 76 51 49 81 69 72 68 52 82 59 77 79 71
 74 73 88 47 75 66 50 63 67 80 58 90 39 78 55 85 65 91 83 57 56 44 60 61
 48 46 23 45 94 27 38 30 35 54 41 43 32 34 93 37 36 95 42 29 24 96 20 33
 97]


> Distribuição dos valores :

 0     69152
80      184
77      165
76      164
78      161
      ...  
23        1
27        1
24        1
20        1
33        1
Name: Metacritic score, Length: 73, dtype: int64


> Descrição:

count    72934.000000
mean         3.774262
std         16.315793
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         97.000000
Name: Metacritic score, dtype: float64



### Limpando Dataset

-------

In [6]:
# ---- Retirando alguns valores nulos ----

df_principal.dropna(subset=['Tags', 'Metacritic score'], inplace=True)
df_principal = df_principal[df_principal['Metacritic score'] != 0]


# ---- Amostragem para anos > 2019 e < 2023 ----

df_principal = df_principal[ df_principal["Release date"].map(lambda date: 
                                                              ((int(date.split(' ')[-1]) > 2020) and 
                                                               int(date.split(' ')[-1]) < 2023))== True]
df_principal.reset_index(inplace=True, drop=True)


print(f"\n> Tamanho do dataset pós limpeza:\n {df_principal.shape}\n")

print(f"\n> Descrição coluna Metacritic score:\n\n {df_principal['Metacritic score'].describe()}\n")


> Tamanho do dataset pós limpeza:
 (299, 39)


> Descrição coluna Metacritic score:

 count    299.000000
mean      76.963211
std        7.544186
min       48.000000
25%       73.000000
50%       78.000000
75%       82.000000
max       97.000000
Name: Metacritic score, dtype: float64



In [7]:
df_principal.head()

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,1345820,Ragnarock,"Jul 15, 2021",50000 - 100000,44,0,16.24,17,Are you ready to rock? Ragnarock is a solo/mul...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,281,0,WanadevStudio,WanadevStudio,"Single-player,Multi-player,PvP,Online PvP,Stea...","Casual,Indie,Racing,Sports","Rhythm,VR,Multiplayer,Music,Racing,PvP,Casual,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,821880,Idol Manager,"Jul 26, 2021",100000 - 200000,102,0,19.99,0,You take on the role of manager at a small (bu...,"['English', 'Japanese', 'Russian', 'Simplified...",...,0,853,0,Glitch Pitch,PLAYISM,"Single-player,Steam Achievements,Steam Worksho...","Indie,Simulation,Strategy","Management,Simulation,Anime,Sexual Content,Str...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1127500,Mini Motorways,"Jul 20, 2021",200000 - 500000,473,0,9.99,0,Have you ever been stuck in traffic and wished...,"['English', 'French', 'Italian', 'German', 'Sp...",...,126,373,186,Dinosaur Polo Club,Dinosaur Polo Club,"Single-player,Steam Achievements,Full controll...","Casual,Indie,Simulation,Strategy","Casual,Simulation,Strategy,Indie,Management,Pu...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1471960,Room to Grow,"Feb 25, 2021",0 - 20000,1,0,9.99,0,Room to Grow is a puzzle game with unique mech...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,91,0,Mischka Kamener,Mischka Kamener,"Single-player,Full controller support,Steam Cl...","Casual,Indie,Strategy","Puzzle,Difficult,Indie,Singleplayer,Sokoban,Ca...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,979310,Disjunction,"Jan 28, 2021",50000 - 100000,0,0,15.99,1,2048. Following the arrest of a prominent comm...,"['English', 'French', 'German', 'Spanish - Spa...",...,0,732,0,Ape Tribe Games,Fireshine Games,"Single-player,Steam Achievements,Full controll...","Action,Indie,RPG","RPG,Choices Matter,Story Rich,Detective,Top-Do...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


## Dataset auxiliar de total_revenue (baixe [aqui](https://drive.google.com/file/d/1DUGhWPNJdPr30eo1oCH9RUPlG_FMHr3L/view?usp=sharing))

In [ ]:
df_total_revenue = pd.read_csv('total_revenue.csv')

In [ ]:
df_total_revenue

In [ ]:
ausentes = []
presentes = []

# Verificando se cada valor da coluna do df1 está presente no df2
for publi in df_principal['Publishers'].tolist():
    if publi.lower() not in [name.lower() for name in df_total_revenue["Name"].tolist()]:
        if publi.lower() not in [name.lower() for name in ausentes]:
            print(publi)
            ausentes.append(publi)
    else:
        if publi.lower() not in [name.lower() for name in presentes]:
            presentes.append(publi)   

In [ ]:
 # Verificar se cada valor da coluna 'Publishers' do df_principal está presente no df_total_revenue
for index, row in df_principal.iterrows():
    publishers = row['Publishers']
    total_revenue = 0
    
    # Verificar se o valor de 'publishers' está presente na coluna 'Name' do df_total_revenue
    for name in publishers.split(','):
        mask = df_total_revenue['Name'].str.lower() == name.lower().strip()
        revenue = df_total_revenue.loc[mask, 'Total revenue'].sum()
        total_revenue += revenue
    
    # Inserir o valor total de receita na coluna 'Total revenue' do df_principal
    df_principal.at[index, 'Total revenue'] = total_revenue

In [ ]:
df_principal

### Tags

-------

In [ ]:

# ---- Tranforma a coluna de Tags em lista ----

df_principal['Tags'] = df_principal.Tags.apply(lambda x: str(x).split(','))
print(f"\n> dataset[Tags] como lista:\n\n{df_principal['Tags']}")


### Montando os Nós da Rede

-------

In [ ]:
id = 0
unique_name_values = df_principal['Name'].unique().tolist()
node_data = []

for _ , linha in  df_principal.iterrows():
    
    if(linha['Name'] in unique_name_values):
        node_data.append([id ,linha['Tags'], linha['Metacritic score']])
        id+=1

df_nodes = pd.DataFrame(node_data,columns=['id','Tags', 'Metacritic score']) 
df_nodes

In [ ]:
# ---- Criando os nós e adiciona atributos aos nós ----

g.add_vp('Metacritic score')

aux = []
for _, infos in df_nodes.iterrows():
    
    #Adiciona vértices:
    infos = infos.astype(object)
    g.add_vertex(infos['id'])
    
    # Adiciona atributos:
    vertice = g.get_vertex(infos['id'])
    vertice['Metacritic score'] = infos['Metacritic score']

### Montando as Arestas da Rede

-------

In [ ]:
def count_tags(lista_tags_1, lista_tags_2):
    '''
    Retorna o número de tags em comum entre duas listas
    '''    
    count_common_tags = 0
        
    for tag1 in lista_tags_1:
        for tag2 in lista_tags_2:
            
            if(tag1 == tag2):
                count_common_tags+=1
    
    return count_common_tags

In [ ]:
node_relation = set()
data_relation = set()

for _, linha1 in df_nodes.iterrows():
    for _,linha2 in df_nodes.iterrows():
        
        proximidade = 0
        
        if(( (linha1['id'] , linha2['id']) not in node_relation) and 
           ( (linha2['id'] , linha1['id']) not in node_relation) and (linha2['id'] != linha1['id'])):
            
            # Calculando TAGS em comum
            proximidade = count_tags(linha1['Tags'], linha2['Tags'])
            
            # Adiciona valor na relação de nós:
            node_relation.add((linha1['id'] , linha2['id']))
        
            data_relation.add((linha1['id'], linha2['id'], proximidade))
            
df_relation = pd.DataFrame(data_relation,columns=['node_1', 'node_2' , 'Tags_em_Comum'] ) 
df_relation

#### Threadhold para a construção das Arestas

In [ ]:
df_relation['Tags_em_Comum'][df_relation['Tags_em_Comum'] != 0].value_counts().plot(kind='bar');

In [ ]:
# ---- Escolhendo threshold par aa criação das arestas a depender da distribuição ----

threshold_proximidade = 5

In [ ]:
# ---- Criando arestas ----

for _ , relacao in df_relation.iterrows():
    
    if(relacao['Tags_em_Comum'] > threshold_proximidade):
        g.add_edge(relacao['node_1'], relacao['node_2'])

### Métricas da Rede

-------

In [ ]:
# ---- Número de arestas, nós e densidade ----

n = g.num_vertices()
m = g.num_edges()

if g.is_directed():
    max_edges = n * (n - 1)
else:
    max_edges = n * (n - 1) // 2
d = m / max_edges

print('\n > Número de vértices de g:', n )
print('\n > Número de arestas de g:', m)
print('\n > Densidade:', d)

### Calculando Degree

-------

In [ ]:
# Propriedade dos vértices
data = gt_data(g)

# Calculando degrees
in_degrees = []
out_degrees = []
degrees = []

for v in g.all_vertices():
    in_degrees.append(v.in_degree())
for v in g.all_vertices():
    out_degrees.append(v.out_degree())
for v in g.all_vertices():
    degrees.append(v.total_degree())
    
data['in_degree'] = in_degrees
data['out_degree'] = out_degrees
data['degree'] = degrees
data['degree'].describe()

In [ ]:
sns.histplot(data['degree'])

### Grafo

-------

In [ ]:
# ----- Salva e renderiza grafo -----
gt_save(g, 'projeto.net.gz')

r = netpixi.render('/projeto.net.gz')

In [ ]:
# ----- Algoritmo Reingold Fruchterm -----
m = gt_draw.fruchterman_reingold_layout(g)
gt_move(g, m)
gt_save(g, 'projeto_frunch.net.gz')
r = netpixi.render('projeto_frunch.net.gz');

In [ ]:
lista_int_coreness = []
lista_float_coreness = []

# Adiciona propriedades ao nó
g.add_vp('core')
g.add_vp('coreness')

# ---- Funções ----

def surprise():
    matrix = spectral.adjacency(g)
    algorithm = cpnet.Surprise()
    algorithm.detect(matrix)
    return algorithm.get_coreness()

def rombach():
    matrix = spectral.adjacency(g)
    algorithm = cpnet.Rombach()
    algorithm.detect(matrix)
    return algorithm.get_coreness()

# ---- Roda Algorítimo ----
c_core = surprise()
c_coreness = rombach()


for i, coreness in c_core.items():
    v = g.get_vertex_by_index(i)
    v['core'] = int(coreness)
    lista_int_coreness.append(int(coreness))
df_nodes['core'] = lista_int_coreness

for i, coreness in c_coreness.items():
    v = g.get_vertex_by_index(i)
    v['coreness'] = float(coreness)
    lista_float_coreness.append(float(coreness))
df_nodes['coreness'] = lista_float_coreness


# ---- Modifica cor ----
for v in g.all_vertices():
    if v['core'] == 1:
        r.vertex(v['id'], color=0xff0000)
    else:
        r.vertex(v['id'], color=0x00ff00)
        
# ---- Modifica tamanho ----

for v in g.all_vertices():
    r.vertex(v['id'], size=(10 + 40 * v['coreness']))
    r.vertex(v['Metacritic score'], color=0xff0000)    

### Regressão Linear

-------

In [ ]:
# Rename Metacritc Score name
df_nodes.rename(columns = {'Metacritic score':'MetacriticScore'}, inplace = True)

In [ ]:
sns.scatterplot(x=df_nodes['coreness'], y=df_nodes['MetacriticScore']);

In [ ]:
sns.regplot(x=df_nodes['coreness'], y=df_nodes['MetacriticScore']);

In [ ]:
result = reg.linear(data=df_nodes, formula=" MetacriticScore ~ coreness")

In [ ]:
result.summary()

In [ ]:
result.micro_summary()

In [ ]:
result.plot_residuals()

In [ ]:
# Observa-se um treshold de divisão no coreness, dessa forma, podemos dividir esses dados em dois conjuntos
# E assim aplicar uma regressão em cada metade.

threshold_coreness = 0.5

df_nodes_menor = df_nodes[df_nodes['coreness'] < threshold_coreness]
df_nodes_maior =  df_nodes[df_nodes['coreness'] > threshold_coreness]

In [ ]:
df_nodes_menor.head()

In [ ]:
df_nodes_maior.head()

In [ ]:
# ------- Para a primeira metade  -------

sns.regplot(x=df_nodes_menor['coreness'], y=df_nodes_menor['MetacriticScore']);

In [ ]:
result_menor = reg.linear(data=df_nodes_menor, formula='MetacriticScore ~ coreness')

In [ ]:
result_menor.micro_summary()

In [ ]:
result_menor.plot_residuals()

In [ ]:
# ------- Para a segunda metade  -------

sns.regplot(x=df_nodes_maior['coreness'], y=df_nodes_maior['MetacriticScore']);

In [ ]:
result_maior = reg.linear(data=df_nodes_maior, formula='MetacriticScore ~ coreness')

In [ ]:
result_maior.micro_summary()

In [ ]:
result_menor.plot_residuals()